In [1]:
import os
import pickle
import matplotlib.pyplot as plt
import numpy as np
import torch

from train_dynamics import generate_data
from policygradient import PolicyGradientOptions, run_policy_gradient, Regularizer
from ltimult import LQRSysMult

In [2]:
device = "cpu"
cached_fn = os.path.join("dynamics_trained", "dynamics.nf")
with open(cached_fn, "rb") as f:
    encoder = pickle.load(f)
encoder.to(device)

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=15, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer): Linear(in_features=12, out_features=50, bias=True)
              (linear_layers): ModuleList(
                (0-1): 2 x Linear(in_features=50, out_features=50, bias=True)
              )
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (final_layer): Linear(in_features=50, out_features=87, bias=True)
        )
      )
      (1): LULinear()
      (2): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=15, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_laye

In [3]:
def calc_scores(xs, Cs, k=10):
    cal_samples = encoder.sample(k, xs).detach().cpu().numpy()
    cal_samples = cal_samples.reshape((cal_samples.shape[0], k, 2, 3))
    cal_tiled = np.transpose(np.tile(Cs.detach().cpu().numpy(), (k, 1, 1, 1)), (1, 0, 2, 3))
    cal_diff = cal_samples - cal_tiled
    cal_norms = np.linalg.norm(cal_diff, ord=2, axis=(2,3))
    return np.min(cal_norms, axis=-1)

In [4]:
# C in R^2x3
cal_sims = 1_000
cal_C_hats, cal_x = generate_data(cal_sims)
cal_C_hats = cal_C_hats.reshape((cal_sims, 2, 3))
cal_scores = calc_scores(cal_x, cal_C_hats)
q_hat = np.quantile(cal_scores, q = 0.95)

/home/yppatel/anaconda3/envs/chig/lib/python3.8/site-packages/nflows/transforms/lu.py:80: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2191.)
  outputs, _ = torch.triangular_solve(


In [5]:
k = 10
test_sims = 1
_, test_x = generate_data(test_sims)
test_C_hats = encoder.sample(k, test_x).detach().cpu().numpy()
test_C_hats = test_C_hats.reshape((test_C_hats.shape[0], k, 2, 3))

In [6]:
# Get system dynamics from side information
C_hat = test_C_hats[0][0]
A = C_hat[:,:2]
B = C_hat[:,2:]

# System problem data
Q = np.eye(A.shape[-1])
R = np.eye(B.shape[-1])
S0 = np.eye(A.shape[-1])

# useless for us, but just keep to use library code
Aa = np.zeros(A.shape)
Aa = Aa[:, :, np.newaxis]
Bb = np.zeros(B.shape)
Bb = Bb[:, :, np.newaxis]
a = np.zeros((1, 1))
b = np.zeros((1, 1))

In [9]:
opt_steps = 5_000

# Start with an initially stabilizing (feasible) controller;
# for this example the system is open-loop mean-square stable
K_star = np.zeros([B.shape[1], A.shape[1]])

for opt_step in range(opt_steps):
    SS = LQRSysMult(A, B, a, Aa, b, Bb, Q, R, S0)
    SS.setK(K_star)
    PGO = PolicyGradientOptions(epsilon=(1e-2) * SS.Kare.size,
                                eta=1e-3,
                                max_iters=1,
                                disp_stride=1,
                                keep_hist=True,
                                opt_method='proximal',
                                keep_opt='last',
                                step_direction='gradient',
                                stepsize_method='constant',
                                exact=True,
                                regularizer=Regularizer('vec1'),
                                regweight=0.0,
                                stop_crit='gradient',
                                fbest_repeat_max=0,
                                display_output=True,
                                display_inplace=True,
                                slow=False)

    # Run (regularized) policy gradient
    run_policy_gradient(SS, PGO)
    K_star = SS.K

    # Print the regularized optimal gains (from proximal gradient optimization)
    # and the unregularized optimal gains (from solving a Riccati equation)
    print('Optimized sparse gains')
    print(SS.K)
    print('Riccati gains')
    print(SS.Kare)

Iteration | Stop quant / threshold |  Curr obj |  Best obj | Norm of gain delta | Stepsize  | Sparsity
                                                                                                        1 |  6.705e-01 / 2.000e-02 | 4.315e+00 | 4.315e+00 |            inf | 1.000e-03 |   0.00%
Max iterations exceeded, stopping optimization
Policy gradient descent optimization completed after 1 iterations, 0.007 seconds
Optimized sparse gains
[[-0.00013031  0.00065771]]
Riccati gains
[[-0.02881471  0.10471411]]
Iteration | Stop quant / threshold |  Curr obj |  Best obj | Norm of gain delta | Stepsize  | Sparsity
                                                                                                        1 |  6.662e-01 / 2.000e-02 | 4.315e+00 | 4.315e+00 |            inf | 1.000e-03 |   0.00%
Max iterations exceeded, stopping optimization
Policy gradient descent optimization completed after 1 iterations, 0.006 seconds
Optimized sparse gains
[[-0.00025991  0.00131118]]
Riccat

KeyboardInterrupt: 